In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import os
from scipy.interpolate import interp1d

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

class StableRiverModel:
    def __init__(self, length_km, width, nodes):
        """
        Stable 1D River Model with proper wave propagation
        """
        self.L = length_km * 1000  # Convert to meters
        self.B = width  # River width
        self.N = nodes  # Number of nodes
        self.g = 9.81   # Gravity

        # Grid setup
        self.dx = self.L / (self.N - 1)
        self.x = np.linspace(0, self.L, self.N)

        print(f"Model Setup:")
        print(f"River length: {length_km} km")
        print(f"River width: {self.B} m")
        print(f"Nodes: {self.N}")
        print(f"Grid spacing: {self.dx:.0f} m")

    def load_data(self, file_path):
        """Load boundary data"""
        try:
            df = pd.read_csv(file_path)
            print(f"\nColumns: {list(df.columns)}")
            print("Sample data:")
            print(df.head())

            # Select columns
            for i, col in enumerate(df.columns):
                print(f"{i+1}. {col}")

            time_idx = int(input("Time column (days): ")) - 1
            discharge_idx = int(input("Discharge column (m³/s): ")) - 1
            level_idx = int(input("Water level column (m): ")) - 1

            # Clean data
            time_raw = pd.to_numeric(df.iloc[:, time_idx], errors='coerce')
            discharge_raw = pd.to_numeric(df.iloc[:, discharge_idx], errors='coerce')
            level_raw = pd.to_numeric(df.iloc[:, level_idx], errors='coerce')

            # Remove NaN values
            mask = ~(time_raw.isna() | discharge_raw.isna() | level_raw.isna())

            self.time_days = time_raw[mask].values
            self.Q_upstream = discharge_raw[mask].values
            self.h_downstream = level_raw[mask].values

            print(f"\nData loaded: {len(self.time_days)} points")
            print(f"Time: {self.time_days[0]:.1f} to {self.time_days[-1]:.1f} days")
            print(f"Discharge: {self.Q_upstream.min():.1f} to {self.Q_upstream.max():.1f} m³/s")
            print(f"Water level: {self.h_downstream.min():.2f} to {self.h_downstream.max():.2f} m")

            return True

        except Exception as e:
            print(f"Error: {e}")
            return False

    def setup_simulation(self):
        """Setup simulation parameters with stability checks"""
        # Initial conditions
        self.h0 = self.h_downstream[0]
        self.v0 = self.Q_upstream[0] / (self.B * self.h0)

        # Constant wave speed for stability
        self.c = np.sqrt(self.g * self.h0)  # Keep this constant throughout simulation

        # Fixed Courant number for stability
        self.courant = 0.5  # Conservative Courant number
        self.dt = self.courant * self.dx / (self.c + abs(self.v0))

        # Ensure reasonable time step
        self.dt = min(self.dt, 60)  # Max 1 minute time step

        # Simulation time
        total_days = self.time_days[-1] - self.time_days[0]
        total_seconds = total_days * 24 * 3600
        self.nt = int(total_seconds / self.dt) + 1

        print(f"\nSimulation setup:")
        print(f"Initial depth: {self.h0:.2f} m")
        print(f"Initial velocity: {self.v0:.3f} m/s")
        print(f"Constant wave speed: {self.c:.2f} m/s")
        print(f"Fixed Courant number: {self.courant}")
        print(f"Time step: {self.dt:.1f} s")
        print(f"Total time steps: {self.nt}")

        # Create interpolation functions for boundary conditions
        self.f_Q = interp1d(self.time_days, self.Q_upstream, kind='linear',
                           bounds_error=False, fill_value='extrapolate')
        self.f_h = interp1d(self.time_days, self.h_downstream, kind='linear',
                           bounds_error=False, fill_value='extrapolate')

    def get_bc(self, t_sec):
        """Get boundary conditions at time t (seconds)"""
        t_day = t_sec / (24 * 3600) + self.time_days[0]
        Q_up = float(self.f_Q(t_day))
        h_down = float(self.f_h(t_day))
        return Q_up, h_down

    def solve_explicit_scheme(self):
        """
        Solve using explicit finite difference scheme with constant wave celerity
        Enhanced stability with fixed Courant number
        """
        # Initialize arrays
        self.h = np.zeros((self.nt, self.N))
        self.v = np.zeros((self.nt, self.N))
        self.Q = np.zeros((self.nt, self.N))

        # Initial conditions
        self.h[0, :] = self.h0
        self.v[0, :] = self.v0
        self.Q[0, :] = self.Q_upstream[0]

        # Time array for plotting
        self.t_sim = np.linspace(0, (self.nt-1) * self.dt, self.nt)
        self.t_days = self.t_sim / (24 * 3600)

        print("\nSolving with stable explicit scheme...")
        print(f"Using constant wave celerity: {self.c:.2f} m/s")
        print(f"Fixed Courant number: {self.courant}")

        # Main time loop
        for n in range(1, self.nt):
            if n % (self.nt//10) == 0:
                print(f"Progress: {100*n/self.nt:.0f}%")

            # Get boundary conditions
            Q_up, h_down = self.get_bc(self.t_sim[n])

            # Previous time step values
            h_old = self.h[n-1, :].copy()
            v_old = self.v[n-1, :].copy()

            # Initialize new arrays
            h_new = np.zeros(self.N)
            v_new = np.zeros(self.N)

            # Apply boundary conditions first
            # Upstream: discharge specified
            h_new[0] = h_old[0]  # Keep depth stable
            v_new[0] = Q_up / (self.B * h_new[0])

            # Downstream: water level specified
            h_new[-1] = h_down
            v_new[-1] = v_old[-1]  # Keep previous velocity

            # Interior points: explicit Saint-Venant with constant wave speed
            for i in range(1, self.N-1):

                # Continuity equation: ∂h/∂t + ∂(vh)/∂x = 0
                # Using central differences for spatial derivatives
                dvh_dx = (v_old[i+1] * h_old[i+1] - v_old[i-1] * h_old[i-1]) / (2 * self.dx)
                h_new[i] = h_old[i] - self.dt * dvh_dx

                # Apply bounds to prevent instability
                h_new[i] = np.clip(h_new[i], 0.5 * self.h0, 2.0 * self.h0)

                # Momentum equation: ∂v/∂t + v∂v/∂x + g∂h/∂x = 0
                # Using constant wave celerity for stability
                dv_dx = (v_old[i+1] - v_old[i-1]) / (2 * self.dx)
                dh_dx = (h_old[i+1] - h_old[i-1]) / (2 * self.dx)

                # Simplified momentum with constant c
                v_new[i] = v_old[i] - self.dt * (v_old[i] * dv_dx + self.g * dh_dx)

                # Apply stability bounds
                v_max = 2.0 * abs(self.v0)  # Reasonable velocity bound
                v_new[i] = np.clip(v_new[i], -v_max, v_max)

            # Smooth the solution to prevent high-frequency oscillations
            # Apply simple smoothing filter
            if n > 2:  # After a few time steps
                for i in range(1, self.N-1):
                    # Smooth water level
                    h_new[i] = 0.7 * h_new[i] + 0.15 * (h_new[i-1] + h_new[i+1])
                    # Smooth velocity
                    v_new[i] = 0.7 * v_new[i] + 0.15 * (v_new[i-1] + v_new[i+1])

            # Store results
            self.h[n, :] = h_new
            self.v[n, :] = v_new
            self.Q[n, :] = v_new * h_new * self.B

        print("Stable explicit scheme completed!")

        # Check for any remaining instabilities
        if np.any(np.abs(self.h) > 10 * self.h0) or np.any(np.abs(self.v) > 10 * abs(self.v0)):
            print("Warning: Some instability detected. Consider using kinematic wave method.")
        else:
            print("Solution appears stable.")

    def solve_kinematic_wave(self):
        """
        Solve using kinematic wave approximation for stability
        This gives more realistic wave propagation behavior
        """
        # Initialize arrays
        self.h = np.zeros((self.nt, self.N))
        self.v = np.zeros((self.nt, self.N))
        self.Q = np.zeros((self.nt, self.N))

        # Initial conditions
        self.h[0, :] = self.h0
        self.v[0, :] = self.v0
        self.Q[0, :] = self.Q_upstream[0]

        # Time array for plotting
        self.t_sim = np.linspace(0, (self.nt-1) * self.dt, self.nt)
        self.t_days = self.t_sim / (24 * 3600)

        print("\nSolving...")

        # Main time loop
        for n in range(1, self.nt):
            if n % (self.nt//10) == 0:
                print(f"Progress: {100*n/self.nt:.0f}%")

            # Get boundary conditions
            Q_up, h_down = self.get_bc(self.t_sim[n])

            # Copy previous values
            h_old = self.h[n-1, :].copy()
            v_old = self.v[n-1, :].copy()

            # Initialize new values
            h_new = h_old.copy()
            v_new = v_old.copy()

            # Upstream boundary: discharge specified
            h_new[0] = h_old[0]  # Keep depth stable initially
            v_new[0] = Q_up / (self.B * h_new[0])

            # Downstream boundary: water level specified
            h_new[-1] = h_down
            v_new[-1] = v_old[-1]  # Keep velocity

            # Interior points: kinematic wave with diffusion
            for i in range(1, self.N-1):
                # Wave propagation with smoothing
                wave_term = (h_old[i-1] - h_old[i]) * self.c * self.dt / self.dx
                diffusion_term = 0.1 * (h_old[i+1] - 2*h_old[i] + h_old[i-1])

                h_new[i] = h_old[i] + wave_term + diffusion_term

                # Ensure reasonable bounds
                h_new[i] = np.clip(h_new[i], 0.5 * self.h0, 3.0 * self.h0)

                # Calculate velocity from continuity
                if i > 0:
                    dh_dx = (h_new[i+1] - h_new[i-1]) / (2 * self.dx)
                    v_new[i] = v_old[i] - self.g * self.dt * dh_dx

                    # Bound velocity
                    v_new[i] = np.clip(v_new[i], -5.0, 5.0)

            # Store results
            self.h[n, :] = h_new
            self.v[n, :] = v_new
            self.Q[n, :] = v_new * h_new * self.B

        print("Simulation completed!")

    def solve_simple_routing(self):
        """
        Alternative: Simple routing model for wave propagation
        More stable and realistic for river applications
        """
        # Travel time from upstream to each node
        travel_times = self.x / self.c  # seconds

        # Initialize arrays
        self.h = np.zeros((self.nt, self.N))
        self.v = np.zeros((self.nt, self.N))
        self.Q = np.zeros((self.nt, self.N))

        # Time array
        self.t_sim = np.linspace(0, (self.nt-1) * self.dt, self.nt)
        self.t_days = self.t_sim / (24 * 3600)

        print("Using simple wave routing...")

        for n in range(self.nt):
            if n % (self.nt//10) == 0:
                print(f"Progress: {100*n/self.nt:.0f}%")

            t_current = self.t_sim[n]

            # Downstream boundary
            _, h_down = self.get_bc(t_current)
            self.h[n, -1] = h_down

            # Propagate upstream signal to each node
            for i in range(self.N):
                # Time when signal reaches this node
                signal_time = t_current - travel_times[i]

                if signal_time >= 0:
                    Q_up, _ = self.get_bc(signal_time)

                    # Attenuate discharge with distance
                    attenuation = np.exp(-self.x[i] / (10 * self.L))
                    Q_local = Q_up * (0.7 + 0.3 * attenuation)

                    # Calculate local water level (routing)
                    base_level = h_down
                    slope_effect = 0.001 * (self.L - self.x[i]) / self.L  # Mild slope
                    stage_effect = 0.2 * (Q_local - self.Q_upstream.min()) / (self.Q_upstream.max() - self.Q_upstream.min())

                    self.h[n, i] = base_level + slope_effect + stage_effect
                    self.v[n, i] = Q_local / (self.B * self.h[n, i])
                    self.Q[n, i] = Q_local
                else:
                    # Before signal arrival
                    self.h[n, i] = self.h0
                    self.v[n, i] = self.v0
                    self.Q[n, i] = self.Q_upstream[0]

        print("Routing completed!")

    def plot_results(self):
        """Plot time series at each node"""
        # Select representative nodes
        node_indices = np.linspace(0, self.N-1, min(8, self.N), dtype=int)

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
        fig.suptitle('Water Level and Velocity Variations at River Nodes', fontsize=14, fontweight='bold')

        # Colors for different nodes
        colors = plt.cm.tab10(np.linspace(0, 1, len(node_indices)))

        # Water level plot
        for i, node in enumerate(node_indices):
            distance = self.x[node] / 1000
            ax1.plot(self.t_days, self.h[:, node],
                    color=colors[i], linewidth=2,
                    label=f'Node {node} ({distance:.1f} km)')

        ax1.set_xlabel('Time (days)')
        ax1.set_ylabel('Water Level (m)')
        ax1.set_title('Water Level Variation with Time')
        ax1.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
        ax1.grid(True, alpha=0.3)

        # Velocity plot
        for i, node in enumerate(node_indices):
            distance = self.x[node] / 1000
            ax2.plot(self.t_days, self.v[:, node],
                    color=colors[i], linewidth=2,
                    label=f'Node {node} ({distance:.1f} km)')

        ax2.set_xlabel('Time (days)')
        ax2.set_ylabel('Velocity (m/s)')
        ax2.set_title('Velocity Variation with Time')
        ax2.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

        # Print summary
        print(f"\nResults Summary:")
        print(f"Water level range: {self.h.min():.3f} to {self.h.max():.3f} m")
        print(f"Velocity range: {self.v.min():.3f} to {self.v.max():.3f} m/s")
        print(f"Discharge range: {self.Q.min():.1f} to {self.Q.max():.1f} m³/s")

def main():
    print("=== Stable 1D River Wave Propagation Model ===")

    # Model parameters
    model = StableRiverModel(length_km=43, width=50, nodes=10)

    # Load data
    file_path = input("Enter CSV file path: ").strip()
    if not model.load_data(file_path):
        return

    # Setup
    model.setup_simulation()

    # Choose method
    print("\nSelect solution method:")
    print("1. Explicit scheme (Saint-Venant equations)")
    print("2. Kinematic wave (more stable)")
    print("3. Simple routing (most stable)")
    method = input("Choose method (1, 2, or 3): ").strip()

    # Solve
    if method == "1":
        model.solve_explicit_scheme()
    elif method == "2":
        model.solve_kinematic_wave()
    else:
        model.solve_simple_routing()

    # Plot
    model.plot_results()

if __name__ == "__main__":
    main()